In [1]:
import numpy as np
import pandas as pd
import random
from joblib import Parallel, delayed

In [2]:

# 生成随机矩阵X_n
n = 1000    # 样本量
p = 1000    # 变量数

# 创建协方差矩阵
cov_matrix = np.full((p, p), 0.5)  # 创建一个所有元素都是 0.5 的矩阵
np.fill_diagonal(cov_matrix, 2.0)  # 将对角线元素设置为 2.0

mean_vector                = np.zeros(p)  # 均值向量
X                          = np.random.multivariate_normal(mean_vector, cov_matrix, n)
# 奇异值分解
U, S, Vt = np.linalg.svd(X)

# 计算条件数
condition_number = S[0] / S[-1]
print("条件数:", condition_number)

print("最大奇异值:", S[0])
print("最小奇异值:", S[-1])

条件数: 31942.033758957987
最大奇异值: 709.0657622865946
最小奇异值: 0.02219851646383476


In [4]:
# 定义β向量
beta = np.zeros(p)  # 初始化β向量为0

# 根据条件设置β向量的值
beta[0:5]   = 2.0   # βi = 2.0, i = 1, 2, 3, 4, 5
beta[5:10]  = -2.0  # βi = -2.0, i = 6, 7, 8, 9, 10
beta[10:15] = 1.0   # βi = 1.0, i = 11, 12, 13, 14, 15
beta[15:20] = -1.0  # βi = -1.0, i = 16, 17, 18, 19, 20
# beta[20:25] = 5.0   # βi = 5.0, i = 21, 22, 23, 24, 25

# Generate 20 random indices
indices = np.random.randint(low=0, high=p, size=20)

# Get the indices of non-zero elements in beta
nonzero_indices = np.nonzero(beta)[0]

# print(nonzero_indices)

# 初始化random_beta为零向量
random_beta = np.zeros(p)

# Assign the non-zero elements randomly
# 此处假设的意图是将beta中的非零元素随机分配到random_beta中
# 但这样的操作逻辑上存在问题，因为indices的长度和nonzero_indices可能不同
# 为了简化，这里我们只是简单地复制beta到random_beta
random_beta[indices] = beta[nonzero_indices]

# Print the randomly assigned beta
# print(random_beta)
beta = random_beta
nonzero_indices = np.nonzero(beta)[0]
print(nonzero_indices)

[ 14  20  79 152 223 251 284 423 544 593 622 637 687 740 746 819 826 866
 937 981]


In [5]:
# 生成均值为0方差为4的n维向量en
mean    = 0  # 均值
std_dev = 2  # 标准差，方差为标准差的平方，即4

# 生成向量
e = np.random.normal(mean, std_dev, n)
norm_e = np.linalg.norm(e)
Y = np.dot(X, beta) + e
print(norm_e)

66.04652964580067


In [6]:
tau   = 1
delta = tau*norm_e
s1, s2 = X.shape

In [80]:
# 最小二乘法
def least_square(X, Y):
    beta = np.linalg.lstsq(X, Y, rcond=None)[0]
    return beta
beta_LS = least_square(X, Y)
norm_LS = np.linalg.norm(beta_LS - beta)
print(norm_LS)
sigma_LS = np.linalg.norm(Y - np.dot(X, beta_LS))
sigma_LS = sigma_LS / np.sqrt(s2)
print(sigma_LS)

53.30149225611223
1.2081041155449896e-13


In [ ]:
# 奇异值分解
U, S, Vt = np.linalg.svd(X)

# 定义一个函数来计算 norm_cut
def compute_norm_cut(k):
    # 截断奇异值
    S_truncated = np.zeros_like(S)
    S_truncated[:k] = S[:k]

    # 构造截断后的矩阵
    Sigma_truncated = np.zeros((U.shape[0], Vt.shape[0]))
    np.fill_diagonal(Sigma_truncated, S_truncated)
    X_truncated = U @ Sigma_truncated @ Vt

    # 计算 S_cut 和 S_inv
    S_cut = Sigma_truncated.T @ Sigma_truncated
    S_inv = np.zeros_like(S_cut)
    S_inv[S_cut != 0] = 1 / S_cut[S_cut != 0]

    # 计算 beta_cut
    beta_cut = Vt.T @ S_inv @ Vt @ X.T @ Y

    # 计算 norm_cut
    norm_cut = np.linalg.norm(beta_cut - beta)
    
    return norm_cut, k

# 使用Parallel和delayed并行处理
results = Parallel(n_jobs=-1)(delayed(compute_norm_cut)(k) for k in range(1000, 0, -1))

# 找到最小的 norm_cut 及其对应的 k 值
min_norm_cut, best_k = min(results, key=lambda x: x[0])

# 输出最小的 norm_cut 及其对应的 k 值
print(f"最小的 norm_cut: {min_norm_cut}, 对应的 k: {best_k}")

In [ ]:
# Spectral cut-off regression

# 奇异值分解
U, S, Vt = np.linalg.svd(X)

# 选择截断点
k = best_k  # 截断点，可以根据需要调整

# 截断奇异值
S_truncated = np.zeros_like(S)
S_truncated[:k] = S[:k]

# 构造截断后的矩阵
Sigma = np.zeros((U.shape[0], Vt.shape[0]))
Sigma_truncated = np.zeros((U.shape[0], Vt.shape[0]))
np.fill_diagonal(Sigma, S)
np.fill_diagonal(Sigma_truncated, S_truncated)
X_truncated = U @ Sigma_truncated @ Vt
S_cut = Sigma_truncated.T@Sigma_truncated
S_inv = np.zeros_like(S_cut)
S_inv[S_cut != 0] = 1 / S_cut[S_cut != 0]
beta_cut = Vt.T @ S_inv @ Vt @ X.T @ Y
norm_cut = np.linalg.norm(beta_cut - beta)
I = np.eye(s2)
beta_cut_debias =(2*I -Sigma.T@ Sigma @ S_inv) @ beta_cut
norm_cut_debias = np.linalg.norm(beta_cut_debias - beta)
print(norm_cut, norm_cut_debias)
sigma_cut = np.linalg.norm(Y - X @ beta_cut)
sigma_cut = sigma_cut / np.sqrt(s2)
print(sigma_cut)
sigma_cut_debias = np.linalg.norm(Y - X @ beta_cut_debias)
sigma_cut_debias = sigma_cut_debias / np.sqrt(s2)
print(sigma_cut_debias)

In [7]:
# Ridge Regression
C = 200000
m = 100
def compute_mse(c, X, Y):
    alpha = c / s1
    I = np.eye(s2)
    beta_r = np.linalg.inv(X.T @ X + alpha * I) @ X.T @ Y
    mse = np.linalg.norm(beta_r - beta)
    return mse

# 假设 X, Y, C 已经定义
mse_list = Parallel(n_jobs=-1)(delayed(compute_mse)(c, X, Y) for c in range(0, C + 1, m))

best_c = np.argmin(mse_list)*m
print("The value of C that minimizes the mean squared error is:", best_c)

The value of C that minimizes the mean squared error is: 95000


In [17]:
# 岭回归
def ridge(X, Y, C):
    alpha  = C/s1
    I      = np.eye(s2)
    gen    = np.linalg.inv(X.T @ X + alpha*I) @ X.T
    beta   = gen @ Y
    beta_debias = (I + alpha*np.linalg.inv(X.T @ X + alpha*I))@beta
    return beta, beta_debias

beta_Ridge, beta_Ridge_debias, gen_Ridge = ridge(X, Y, best_c)
norm_Ridge = np.linalg.norm(beta_Ridge - beta)
norm_Ridge_debias = np.linalg.norm(beta_Ridge_debias - beta)
print(norm_Ridge, norm_Ridge_debias)
sigma_Ridge = np.linalg.norm(Y - np.dot(X, beta_Ridge))
sigma_Ridge = sigma_Ridge / np.sqrt(s2)
print(sigma_Ridge)
sigma_Ridge_debias = np.linalg.norm(Y - np.dot(X, beta_Ridge_debias))
sigma_Ridge_debias = sigma_Ridge_debias / np.sqrt(s2)
print(sigma_Ridge_debias)

3.211954981423207 3.4416709271979413
1.0130884813345848
0.6417268117639816


In [ ]:
# Lasso Regression
from sklearn.linear_model import Lasso

C = 1000
m = 1
def compute_mse(c, X, Y):
    alpha = c / s1
    lasso =  Lasso(alpha=alpha)
    lasso.fit(X, Y)
    beta_lasso = lasso.coef_
    mse = np.linalg.norm(beta_lasso - beta)
    return mse

# 假设 X, Y, C 已经定义
mse_list = Parallel(n_jobs=-1)(delayed(compute_mse)(c, X, Y) for c in range(0, C + 1, m))

best_c = np.argmin(mse_list)*m
print("The value of C is:", best_c)
lasso =  Lasso(alpha=best_c)
lasso.fit(X, Y)
beta_lasso = lasso.coef_
norm_Lasso = np.linalg.norm(beta_lasso - beta)
print(norm_Lasso)
sigma_Lasso = np.linalg.norm(Y - np.dot(X, beta_lasso))
sigma_Lasso = sigma_Lasso / np.sqrt(s2)
print(sigma_Lasso)

In [97]:
# Landweber回归
t        = 5e-7    # 步长
max_iter = 10000   # 最大迭代次数

def landweber(X, Y, t, max_iter):
    s = X.shape
    m = s[1]
    beta_cur = np.zeros(m)
    for k in range(1, max_iter):
        beta_prev = beta_cur
        beta_cur = beta_prev + t * np.dot(X.T, (Y - np.dot(X, beta_prev)))
        if np.linalg.norm(Y - np.dot(X, beta_cur)) <= 1.5*delta:
            break
    return beta_cur, k
def landweber_debias(X, Y, t, max_iter):
    s = X.shape
    m = s[1]
    beta_cur = np.zeros(m)
    for k in range(1, max_iter):
        beta_prev = beta_cur
        beta_cur = beta_prev + t * np.dot(X.T, (Y - np.dot(X, beta_prev)))
        if k == max_iter:
            break
    return beta_cur
beta_Landweber, k_Landweber = landweber(X, Y, t, max_iter)
beta_Landweber_debias = landweber_debias(X, Y, t, 2*k_Landweber)
norm_Landweber    = np.linalg.norm(beta_Landweber - beta)
norm_Landweber_debias  = np.linalg.norm(beta_Landweber_debias - beta)  

print(norm_Landweber, k_Landweber, norm_Landweber_debias)
sigma_Landeweber = np.linalg.norm(Y - np.dot(X, beta_Landweber))
sigma_Landeweber = sigma_Landeweber / np.sqrt(s2)
print(sigma_Landeweber)
sigma_Landeweber_debias = np.linalg.norm(Y - np.dot(X, beta_Landweber_debias))
sigma_Landeweber_debias = sigma_Landeweber_debias / np.sqrt(s2)
print(sigma_Landeweber_debias)

4.643950876190539 1217 4.101288265260092
3.0207964006327366
2.0200968999970437


In [ ]:
# Showalter回归
t        = 5e-7    # 步长
max_iter = 10000   # 最大迭代次数

def showalter(X, Y, t, max_iter):
    s = X.shape
    m = s[1]
    beta_cur = np.zeros(m)
    Z1 = np.dot(X.T, Y)
    Z2 = np.dot(X.T, X)
    for k in range(1, max_iter):
        beta_prev = beta_cur
        K1 = Z1 - np.dot(Z2, beta_prev)
        K2 = Z1 - np.dot(Z2, (beta_prev + 0.5 * t * K1))
        K3 = Z1 - np.dot(Z2, (beta_prev + 0.5 * t * K2))
        K4 = Z1 - np.dot(Z2, (beta_prev + t * K3))
        beta_cur = beta_prev + t / 6 * (K1 + 2 * K2 + 2 * K3 + K4)
        # print(np.linalg.norm(Y - np.dot(X, beta_cur)))
        # if np.linalg.norm(beta_cur - beta_prev) <= 1e-4:
        if np.linalg.norm(Y - np.dot(X, beta_cur)) <= delta:
            break
    return beta_cur, k
def showalter_debias(X, Y, t, max_iter):
    s = X.shape
    m = s[1]
    beta_cur = np.zeros(m)
    Z1 = np.dot(X.T, Y)
    Z2 = np.dot(X.T, X)
    for k in range(1, max_iter):
        beta_prev = beta_cur
        K1 = Z1 - np.dot(Z2, beta_prev)
        K2 = Z1 - np.dot(Z2, (beta_prev + 0.5 * t * K1))
        K3 = Z1 - np.dot(Z2, (beta_prev + 0.5 * t * K2))
        K4 = Z1 - np.dot(Z2, (beta_prev + t * K3))
        beta_cur = beta_prev + t / 6 * (K1 + 2 * K2 + 2 * K3 + K4)
        if k == max_iter:
            break
    return beta_cur
beta_showalter, k_showalter = showalter(X, Y, t, max_iter)
beta_showalter_debias = showalter_debias(X, X@beta_showalter, t, k_showalter)
beta_showalter_debias = 2*beta_showalter-beta_showalter_debias
norm_showalter = np.linalg.norm(beta_showalter - beta)
norm_showalter_debias = np.linalg.norm(beta_showalter_debias - beta)
print(norm_showalter, k_showalter, norm_showalter_debias)
sigma_showalter = np.linalg.norm(Y - np.dot(X, beta_showalter))
sigma_showalter = sigma_showalter / np.sqrt(s2)
print(sigma_showalter)
sigma_showalter_debias = np.linalg.norm(Y - np.dot(X, beta_showalter_debias))
sigma_showalter_debias = sigma_showalter_debias / np.sqrt(s2)
print(sigma_showalter_debias)

In [ ]:
# Showalter回归
t        = 5e-7    # 步长
max_iter = 10000   # 最大迭代次数

def showalter(X, Y, t, max_iter):
    s = X.shape
    m = s[1]
    beta_cur = np.zeros(m)
    Z1 = np.dot(X.T, Y)
    Z2 = np.dot(X.T, X)
    for k in range(1, max_iter):
        beta_prev = beta_cur
        K1 = Z1 - np.dot(Z2, beta_prev)
        K2 = Z1 - np.dot(Z2, (beta_prev + 0.5 * t * K1))
        K3 = Z1 - np.dot(Z2, (beta_prev + 0.5 * t * K2))
        K4 = Z1 - np.dot(Z2, (beta_prev + t * K3))
        beta_cur = beta_prev + t / 6 * (K1 + 2 * K2 + 2 * K3 + K4)
        # print(np.linalg.norm(Y - np.dot(X, beta_cur)))
        # if np.linalg.norm(beta_cur - beta_prev) <= 1e-4:
        if np.linalg.norm(Y - np.dot(X, beta_cur)) <= delta:
            break
    return beta_cur, k
def showalter_debias(X, Y, t, max_iter):
    s = X.shape
    m = s[1]
    beta_cur = np.zeros(m)
    Z1 = np.dot(X.T, Y)
    Z2 = np.dot(X.T, X)
    for k in range(1, max_iter):
        beta_prev = beta_cur
        K1 = Z1 - np.dot(Z2, beta_prev)
        K2 = Z1 - np.dot(Z2, (beta_prev + 0.5 * t * K1))
        K3 = Z1 - np.dot(Z2, (beta_prev + 0.5 * t * K2))
        K4 = Z1 - np.dot(Z2, (beta_prev + t * K3))
        beta_cur = beta_prev + t / 6 * (K1 + 2 * K2 + 2 * K3 + K4)
        if k == max_iter:
            break
    return beta_cur
beta_showalter, k_showalter = showalter(X, Y, t, max_iter)
beta_showalter_debias = showalter_debias(X, X@beta_showalter, t, k_showalter)
beta_showalter_debias = 2*beta_showalter-beta_showalter_debias
norm_showalter = np.linalg.norm(beta_showalter - beta)
norm_showalter_debias = np.linalg.norm(beta_showalter_debias - beta)
print(norm_showalter, k_showalter, norm_showalter_debias)
sigma_showalter = np.linalg.norm(Y - np.dot(X, beta_showalter))
sigma_showalter = sigma_showalter / np.sqrt(s2)
print(sigma_showalter)
sigma_showalter_debias = np.linalg.norm(Y - np.dot(X, beta_showalter_debias))
sigma_showalter_debias = sigma_showalter_debias / np.sqrt(s2)
print(sigma_showalter_debias)

In [ ]:
# Acceleration regression of order kappa
t        = 5e-5   # 步长
max_iter = 1000   # 最大迭代次数
kappa    = 1.5       # 加速阶数

def ARk(X, Y, t, k_max):
    s = X.shape
    m = s[1]
    beta_prev = np.zeros(m)
    I      = np.eye(m)
    v_prev = np.zeros(m)
    for k in range(1, k_max):
        beta_cur = beta_prev + k*t*v_prev
        D = np.linalg.inv((t*(k*t)**kappa)*X.T @ X + I + (((k*t)**(-kappa))-kappa)/k*I)
        v_cur = D@v_prev+ t*D@np.dot(X.T, (Y - np.dot(X, beta_prev)))
        beta_prev = beta_cur
        v_prev = v_cur
        print(np.linalg.norm(Y - np.dot(X, beta_cur)),k)
        if np.linalg.norm(Y - np.dot(X, beta_prev)) <= delta:
            break
    return beta_cur, k

def ARk_debias(X, Y, t, k_max):
    s = X.shape
    m = s[1]
    beta_prev = np.zeros(m)
    I      = np.eye(m)
    v_prev = np.zeros(m)
    for k in range(1, k_max):
        beta_cur = beta_prev + k*t*v_prev
        D = np.linalg.inv((t*(k*t)**kappa)*X.T @ X + I + (((k*t)**(-kappa))-kappa)/k*I)
        v_cur = D@v_prev+t*D@np.dot(X.T, (Y - np.dot(X, beta_prev)))
        beta_prev = beta_cur
        v_prev = v_cur
        if k == max_iter:
            break
    return beta_cur    

beta_ARk, k_ARk = ARk(X, Y, t, max_iter)
beta_ARk_debias = ARk_debias(X, X@beta_ARk, t, k_ARk)
beta_ARk_debias = 2*beta_ARk - beta_ARk_debias
norm_ARk = np.linalg.norm(beta_ARk - beta)
norm_ARk_debias = np.linalg.norm(beta_ARk_debias - beta)
print(norm_ARk, k_ARk, norm_ARk_debias)
sigma_ARk = np.linalg.norm(Y - np.dot(X, beta_ARk))
sigma_ARk = sigma_ARk / np.sqrt(s2)
print(sigma_ARk)
sigma_ARk_debias = np.linalg.norm(Y - np.dot(X, beta_ARk_debias))
sigma_ARk_debias = sigma_ARk_debias / np.sqrt(s2)
print(sigma_ARk_debias)

In [ ]:
# 二阶渐进回归
t        = 5e-4    # 步长
max_iter = 10000   # 最大迭代次数

def second(X, Y, t, k_max):
    r = X.shape
    m = r[1]
    s = 0.5
    beta_cur = np.zeros(m)
    v_cur    = np.zeros(m)
    q_cur    = np.zeros(m)
    for k in range(1, k_max):
        p        =  2*k/(2*k+2*s+1)
        q_half   =  p*q_cur + p*(t / 2) * np.dot(X.T, (Y - np.dot(X, beta_cur))) 
        beta_cur = beta_cur + t * q_half
        a        = (2*k-2*s+1)*(k+1)/(2*k+2*s+3)/k
        v_cur   =  beta_cur + 2*t*a*q_half
        q_cur    =  (2*k+1-2*s)/(2*k+2)*q_half +(t / 2) * np.dot(X.T, (Y - np.dot(X, v_cur))) 
        # print(np.linalg.norm(Y - np.dot(X, beta_cur)))
        if np.linalg.norm(Y - np.dot(X, beta_cur)) <= delta:
            break
    return beta_cur, k

def second_debias(X, Y, t, k_max):
    r = X.shape
    m = r[1]
    s = 0.5
    beta_cur = np.zeros(m)
    v_cur    = np.zeros(m)
    q_cur    = np.zeros(m)

    for k in range(1, k_max):
        p        =  2*k/(2*k+2*s+1)
        q_half   =  p*q_cur + p*(t / 2) * np.dot(X.T, (Y - np.dot(X, beta_cur))) 
        beta_cur = beta_cur + t * q_half
        a        = (2*k-2*s+1)*(k+1)/(2*k+2*s+3)/k
        v_cur   =  beta_cur + 2*t*a*q_half
        q_cur    =  (2*k+1-2*s)/(2*k+2)*q_half +(t / 2) * np.dot(X.T, (Y - np.dot(X, v_cur))) 
        if k == k_max:
            break
    return beta_cur

beta_second, k_second = second(X, Y, t, max_iter)
beta_second_debias = second_debias(X, X@beta_second, t, k_second)
beta_second_debias = 2*beta_second - beta_second_debias
norm_second = np.linalg.norm(beta_second - beta)
norm_second_debias = np.linalg.norm(beta_second_debias - beta)
print(norm_second, k_second, norm_second_debias)
sigma_second = np.linalg.norm(Y - np.dot(X, beta_second))
sigma_second = sigma_second / np.sqrt(s2)
print(sigma_second)
sigma_second_debias = np.linalg.norm(Y - np.dot(X, beta_second_debias))
sigma_second_debias = sigma_second_debias / np.sqrt(s2)
print(sigma_second_debias)

In [ ]:
# Nesterov回归
t        = 5e-7    # 步长
max_iter = 10000   # 最大迭代次数
omega    = 3       # omega  

def Nesterov(X, Y, t, k_max):
    s = X.shape
    m = s[1]
    beta_cur = np.zeros(m)
    for k in range(1, k_max):
        beta_prev = beta_cur
        z_k = beta_cur + (k - 1) / (k + omega) * (beta_cur - beta_prev)
        XTX_z_k = np.dot(X.T, (Y - np.dot(X, z_k)))
        beta_cur = z_k + t * XTX_z_k
        # print(np.linalg.norm(Y - np.dot(X, beta_cur)))
        if np.linalg.norm(Y - np.dot(X, beta_cur)) <= delta:
            break
    return beta_cur, k+1

def Nesterov_debias(X, Y, t, k_max):
    s = X.shape
    m = s[1]
    beta_cur = np.zeros(m)
    for k in range(1, k_max):
        beta_prev = beta_cur
        z_k = beta_cur + (k - 1) / (k + omega) * (beta_cur - beta_prev)
        XTX_z_k = np.dot(X.T, (Y - np.dot(X, z_k)))
        beta_cur = z_k + t * XTX_z_k
        if k == k_max:
            break
    return beta_cur

beta_Nesterov, k_Nesterov = Nesterov(X, Y, t, max_iter)
beta_Nesterov_debias = Nesterov_debias(X, X@beta_Nesterov, t, k_Nesterov)
beta_Nesterov_debias = 2*beta_Nesterov - beta_Nesterov_debias
norm_Nesterov = np.linalg.norm(beta_Nesterov - beta)
norm_Nesterov_debias = np.linalg.norm(beta_Nesterov_debias - beta)
print(norm_Nesterov, k_Nesterov, norm_Nesterov_debias)
sigma_Nesterov = np.linalg.norm(Y - np.dot(X, beta_Nesterov))
sigma_Nesterov = sigma_Nesterov / np.sqrt(s2)
print(sigma_Nesterov)
sigma_Nesterov_debias = np.linalg.norm(Y - np.dot(X, beta_Nesterov_debias))
sigma_Nesterov_debias = sigma_Nesterov_debias / np.sqrt(s2)
print(sigma_Nesterov_debias)

In [ ]:
from scipy.special import gamma

# 分数阶回归
t        = 5e-5     # 步长
max_iter = 500      # 最大迭代次数
theta    = 1.5       # theta
A        = np.zeros((max_iter+1, max_iter+1))
B        = np.zeros((max_iter, max_iter+1))
g        = 1/gamma(theta)
k1       = t**(theta)/(theta*(theta+1))
k2       = t**(theta)/theta

for k in range(max_iter):
    for j in range(k+2):
        if j == 0:
            d = k**(theta+1)-(k-theta)*(k+1)**(theta)
            A[j, k+1] = k1*d
        elif j < k+1:
            d = (k-j+2)**(theta+1)+(k-j)**(theta+1)-2*(k-j+1)**(theta+1)
            A[j, k+1] = k1*d
        elif j == k+1:
            A[j, k+1] = k1 
    for i in range(k+1):
        if i < k+1:
            d = (k-i+1)**theta-(k-i)**theta  
            B[i, k+1] = k2*d


def Frac(X, Y, k_max):
    s  = X.shape
    m  = s[1]
    b  = np.zeros((m, k_max+1))  
    bp = np.zeros((m, k_max+1))
    for k in range(1, k_max):
        bp[:, k+1] = g*sum([B[j, k+1]*np.dot(X.T, (Y - np.dot(X, b[:,j]))) for j in range(1,k+1)]) 
        b[:, k+1]  = g*(A[k+1, k+1]*np.dot(X.T, (Y - np.dot(X, bp[:,k+1])))+sum([A[j, k+1]*np.dot(X.T, (Y - np.dot(X, b[:,j]))) for j in range(1, k+1)]))
        beta_cur   = b[:, k+1]
        # print(np.linalg.norm(Y - np.dot(X, beta_cur)), k)
        if np.linalg.norm(Y - np.dot(X, beta_cur)) <= delta:
            break
    return beta_cur, k
def Frac_debias(X, Y, k_max):
    s  = X.shape
    m  = s[1]
    b  = np.zeros((m, k_max+1))  
    bp = np.zeros((m, k_max+1))
    for k in range(1, k_max):
        bp[:, k+1] = g*sum([B[j, k+1]*np.dot(X.T, (Y - np.dot(X, b[:,j]))) for j in range(1,k+1)]) 
        b[:, k+1]  = g*(A[k+1, k+1]*np.dot(X.T, (Y - np.dot(X, bp[:,k+1])))+sum([A[j, k+1]*np.dot(X.T, (Y - np.dot(X, b[:,j]))) for j in range(1, k+1)]))
        beta_cur   = b[:, k+1]
        if k == k_max:
            break
    return beta_cur

beta_Frac, k_Frac = Frac(X, Y, max_iter)
beta_Frac_debias = Frac_debias(X, X@beta_Frac, k_Frac)
beta_Frac_debias = 2*beta_Frac - beta_Frac_debias
norm_Frac = np.linalg.norm(beta_Frac - beta)
norm_Frac_debias = np.linalg.norm(beta_Frac_debias - beta)
print(norm_Frac, k_Frac, norm_Frac_debias)
sigma_Frac = np.linalg.norm(Y - np.dot(X, beta_Frac))
sigma_Frac = sigma_Frac / np.sqrt(s2)
print(sigma_Frac)
sigma_Frac_debias = np.linalg.norm(Y - np.dot(X, beta_Frac_debias))
sigma_Frac_debias = sigma_Frac_debias / np.sqrt(s2)
print(sigma_Frac_debias)

